# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof Luis Eduardo Falcón Morales

### **Semana 8**
#### **Actividad en clase: Aprendizaje No Supervisado : Reducción de Dimensionalidad**

En esta Actividad de la semana 10 estaremos viendo el tema de reducción de dimensionalidad mediante la factorización de la Descomposición en Valores Singulares SVD (Singular Value Descomposition).

En particular veremos como dicha factorización de matrices SVD nos permite reducir la dimensión de una matriz de utilidad mediante su proyección a un espacio latente de menor dimensión.

Este tipo de solución suele aplicarse como un primer sistema de recomendación de productos a usuarios, de acuerdo al historial de evaluaciones que han realizado en el pasado a ciertos productos.

Veremos el ejemplo clásico de recomendaciones de películas a usuarios con base a las evaluaciones que ellos mismos han hecho en el pasado.

Usaremos el archivo con 100 mil registros llamado **ml-100k.zip** de la liga: https://grouplens.org/datasets/movielens/

Y en particular de dicho archivo usaremos los archivos llamados **u.data** y **u.item**.

El primer archivo contiene las evaluaciones (rating) de los usuarios (user_id) que han hecho de las películas (item_id) que han visto. El segundo archivo contiene la relación de los IDs de las películas y sus nombres.

Con esta información podrás construir un primer sistema de recomendación, el cual te premitirá empezar a adentrarte en una de las principales áreas del aprendizaje no supervisado y en particular del tema de reducción de dimensionalidad.



In [1]:
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos los archivos correspondientes. En el archvo **README.txt** de la liga dada previamente encuentras la información de cada archivo, en particular los nombres de sus columnas, ya que no están incluídas en cada archivo y que en el caso del archivo u.data, los datos están separados por tabuladores "\t".

In [3]:
usuario_peliculas = pd.read_csv("u.data", header=None, sep="\t")

print(usuario_peliculas.shape)

usuario_peliculas.columns=['user_id', 'item_id', 'rating', 'timestamp']

usuario_peliculas = usuario_peliculas[['user_id', 'item_id', 'rating']]  # seleccionemos las columnas con las que trabajaremos.

usuario_peliculas.head(3)

(100000, 4)


,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1


**Observa que ya tenemos la información primordial para generar nuestro sistema de recomendación, a saber: usuarios con sus evaluaciones a películas que han visto.**

Por ejemplo, en particular podemos ver las películas que ha visto el usuario con ID, 196. Veamos

In [4]:
idx = (usuario_peliculas['user_id']==196)

print(idx.sum())  # total de películas que ha visto el usuario con id 196.

usuario_peliculas[idx]    # id de películas que ha visto este usuario y la manera en que las ha calificado.

39


,user_id,item_id,rating
0,196,242,3
940,196,393,4
1133,196,381,4
1812,196,251,3
1896,196,655,5
2374,196,67,5
6910,196,306,4
7517,196,238,4
7842,196,663,5
10017,196,111,4


Sin embargo, para que las recomendaciones nos den mayor información, extraemos los nombres asociados a cada uno de los IDs de las películas. Dicha información y mucha otra más, se encuentra en el segundo archivo, **u.item**.

Así, del segundo archivo, **u.item** solamente necesitamos el nombre de la película con base a su id, o item_id. En este caso la información está separada por el caracter "|". El argumento "latin-1" ayuda a reconocer caracteres de lenguas de origen latino como el inglés, en este caso.

In [5]:
columnas = ['item_id', 'movie_title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

peliculasID = pd.read_csv('u.item', sep='|', names=columnas, encoding='latin-1')

nombres_peliculas = peliculasID[['item_id', 'movie_title']]    # DataFrame
nombres_peliculas.head()



,item_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


En la siguiente página encuentras la información del archivo **u.item**, en paricular la relación entre el nombre de una película y su id:

http://files.grouplens.org/datasets/movielens/ml-100k/u.item

Para este ejemplo veamos el caso para la película Titanic, cuyo nombre completo en esta lista es "Titanic (1997)" y su id es el 313, el cual lo puedes obtener también como sigue:

In [6]:
idx = nombres_peliculas['movie_title']=="Titanic (1997)"
nombres_peliculas[idx]

,item_id,movie_title
312,313,Titanic (1997)


Finalmente conjuntemos la información de ambos DataFrame, a través de la información de la columna "item_id". Veamos:

In [7]:
tabla_relacional = pd.merge(usuario_peliculas, nombres_peliculas, on='item_id')   # dataFrame

tabla_relacional.head()

,user_id,item_id,rating,movie_title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


In [8]:
tabla_relacional.shape    # Ya tenemos la relación que necesitamos para generar nuestro primer sistema de recomendación.

(100000, 4)

Veamos de nuevo las películas que ha visto el usuario 196 y la manera en que las ha calificado:

In [9]:
idx = (tabla_relacional['user_id']==196)

print(idx.sum())  # total de películas que ha visto el usuario con id 196.

tabla_relacional[idx]

39


,user_id,item_id,rating,movie_title
0,196,242,3,Kolya (1996)
1518,196,257,2,Men in Black (1997)
13157,196,111,4,"Truth About Cats & Dogs, The (1996)"
13622,196,25,4,"Birdcage, The (1996)"
14668,196,382,4,"Adventures of Priscilla, Queen of the Desert, ..."
28289,196,202,3,Groundhog Day (1993)
34307,196,153,5,"Fish Called Wanda, A (1988)"
42467,196,286,5,"English Patient, The (1996)"
42983,196,66,3,While You Were Sleeping (1995)
44286,196,845,4,That Thing You Do! (1996)


Ahora ya podemos generar nuestra matriz de utilidad, matriz que nos proporciona la evaluación que cada usuario asigna a cada película que ha visto. Le indicamos que rellene con ceros aquellas celdas que no tienen registros (de lo contrario les asigna NaN).

In [10]:
UtMx = tabla_relacional.pivot_table(values='rating', index='user_id', columns='movie_title', fill_value=0)

UtMx.head(10)

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0
6,0,0,0,4,0,0,0,5,0,0,...,0,0,0,4,0,0,0,0,0,0
7,0,0,0,4,0,0,5,5,0,4,...,0,0,0,5,3,0,3,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [11]:
UtMx.columns

Index([''Til There Was You (1997)', '1-900 (1994)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)', '187 (1997)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '3 Ninjas: High Noon At Mega Mountain (1998)', '39 Steps, The (1935)',
       ...
       'Yankee Zulu (1994)', 'Year of the Horse (1997)', 'You So Crazy (1994)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Poisoner's Handbook, The (1995)',
       'Zeus and Roxanne (1997)', 'unknown',
       'Á köldum klaka (Cold Fever) (1994)'],
      dtype='object', name='movie_title', length=1664)

In [12]:
UtMx.shape   # Tenemos 943 usuarios y 1664 películas

(943, 1664)

Observa que tenemos lo que se llama una matriz dispersa (sparse matrix), es decir, una matriz donde existe una gran cantidad de valores cero. Y así sucede en general, porque siempre habrá mucho más productos que los que un usuario estándar pueda adquirir. En este caso, tenermos 943 usuarios con un catálogo de 1664 películas.

Veamos la proporción de datos no cero, apoyándonos en la función count_nonzero() de NumPy:

In [13]:
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))

Total de elementos de la matriz de utilidad: 1569152
Total de elementos diferentes de cero: 99693
Porcentaje de elementos diferentes de cero: 6.4%


Existe toda un área de estudio dentro del Álgebra Lineal referente a el manejo de matrices dispersas, la cual se centra en optimizar la manera en que se representan y las operaciones entre ellas. Sin embargo, por el momento no abordaremos dicho tema.

Para este ejercicio queremos obtener la información latente en relación a las películas, así que necesitamos que los renglones de la matriz de utilidad sean las película. Así, antes de aplicar la factorización SVD para reducción de la dimensionalidad debemos tomar la traspuesta de dicha matriz de utilidad, ya que por el momento lo que tenemos son usuarios en los renglones y películas en las columnas.

Recuerda que si deseas obtener la información latente de los usuarios (para agruparlos por similitud de sus gustos), entonces puedes dejar la matriz en el orden que ya tiene.

En este ejercicio usaremos lo relacionado a películas para ver de manera explícita la información generada y tener un mejor contexto de lo que estamos obteniendo. Recuerda que de los usuarios solamente tenemos su número de ID y aunque podemos obtener la relación entre usuarios similares, solamente tendrías dicha relación entre números de ID.

In [14]:
X = UtMx.T     # Tomamos la traspuesta para que los renglones sean las películas y las columnas los usuarios.
X.shape

(1664, 943)

In [15]:
# Obtengamos la cantidad de información contenida en la factorización SVD, considerando
# los valores singulares más representativos de dicha factorización.
# Para este caso inicial usaremos todos los vectores singulares, para luego
# compararlo con el caso al reducir su dimensión.

 # El número de componentes debe ser menor que el número de características (features/usuarios), que son 943:
SVD = TruncatedSVD(n_components=942, random_state=1)
SVD.fit(X)

num_sv = 10   # En particular los 10 primeros vectores y valores singulares de mayor magnitud tienen esta cantidad de información.

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 10 vectores singulares:
90.8%


**Vemos que usando los primeros 10 valores singulares de mayor magnitud (de los 942 que se tienen), estamos reduciendo la cantidad de información en casi un 90%.**

**Una vez identificada la reducción de dimensionalidad con la cual se deiseñará el sistema de recomendación, obtenemos la representación truncada SVD con dicha dimensión:**

### **En caso de que las recomendaciones obtenidas no las consideremos del todo adecuadas, habrá que aumentar la cantidad de vectores singulares a utilizar.**

In [20]:
# Usemos las primeras 10 componentes de dicha factorización.
# Es decir reducimos la dimensión de la matriz X con solamente sus primeras componentes:

num_sv = 10

SVD = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(1664, 10)

In [21]:
pd.DataFrame(resultant_matrix)

,0,1,2,3,4,5,6,7,8,9
0,1.039994,0.659902,0.045683,0.813836,0.166888,-0.983452,-0.322568,0.308241,-0.052731,-0.750212
1,0.436584,-0.257258,0.352957,-0.668659,-0.293043,-0.005752,-0.233031,-0.566468,0.342563,-0.218086
2,12.543744,5.669181,-4.907831,5.021757,7.880001,-0.812802,-0.094839,-4.844693,2.050889,4.036758
3,25.663725,-12.267635,6.077473,3.113930,0.713252,1.883231,-3.656097,-0.361711,1.416672,-6.075281
4,3.636418,4.217256,2.633297,1.654332,-3.200479,-1.943994,-0.669065,-0.813077,2.391323,-0.343458
...,...,...,...,...,...,...,...,...,...,...
1659,6.566454,1.511652,-5.562123,-1.528528,-3.108552,-2.727393,-0.575903,-0.003038,-0.933451,-1.318181
1660,4.787711,2.142409,3.589151,-4.735094,-1.221921,-2.299773,0.177307,-3.081604,1.512588,1.935139
1661,0.358930,0.371252,0.022971,0.246657,0.650112,-0.005035,-0.017975,-0.513889,0.017669,-0.057135
1662,1.424280,0.814961,-0.490238,-0.654562,-1.029050,-0.322891,0.722481,-0.760222,0.407421,0.147536


In [22]:
# Obtengamos la matriz de correlación de Pearson entre todas las
# variables latentes de las películas:

corrMx = np.corrcoef(resultant_matrix, )
corrMx.shape

(1664, 1664)

In [23]:
# Veamos dónde quedó el índice de la película Titanic en la matriz de utilidad:

me_gusta = "Johnny Mnemonic (1995)"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

id_megusta

785

In [24]:
# Veamos la correlación de esta película, con todas las que se tienen en el catálogo:

corr_recomienda = corrMx[id_megusta]
print(corr_recomienda.shape)
print(corr_recomienda)

(1664,)
[0.46218415 0.19707819 0.7830941  ... 0.41264075 0.84150877 0.14912932]


Veamos qué películas están altamente correlacionadas con la película que seleccionamos. En particular se omiten las correlaciones iguales a uno o muy cercanas a ellas, la cual en particular estará la película correlacionada consigo mismo. Por ejemplo, veamos cuál es el valor de correlación de Pearson de la película seleccionada, consigo misma:

In [25]:
corr_recomienda[id_megusta]

1.0

Seleccionemos entonces qué película se recomendaría a alguien que ha visto y le ha gustado la película Titanic:

In [26]:
list(names[(corr_recomienda > .90) & (corr_recomienda < 0.99)])

['Addams Family Values (1993)',
 'Barb Wire (1996)',
 'Batman & Robin (1997)',
 'Batman Forever (1995)',
 'Batman Returns (1992)',
 'Beverly Hills Cop III (1994)',
 'Coneheads (1993)',
 'Congo (1995)',
 'Cops and Robbersons (1994)',
 'Cowboy Way, The (1994)',
 'Crow, The (1994)',
 'Crow: City of Angels, The (1996)',
 'Cutthroat Island (1995)',
 'Demolition Man (1993)',
 'Highlander III: The Sorcerer (1994)',
 'Hot Shots! Part Deux (1993)',
 'Hunted, The (1995)',
 'Judge Dredd (1995)',
 'Last Action Hero (1993)',
 'Love Bug, The (1969)',
 'Mortal Kombat (1995)',
 'Naked Gun 33 1/3: The Final Insult (1994)',
 'Pest, The (1997)',
 'Quick and the Dead, The (1995)',
 'Rising Sun (1993)',
 "Robert A. Heinlein's The Puppet Masters (1994)",
 'Robocop 3 (1993)',
 'Shadow, The (1994)',
 'Species (1995)',
 'Star Trek V: The Final Frontier (1989)',
 'Stargate (1994)',
 'Striking Distance (1993)',
 'Three Musketeers, The (1993)',
 'Timecop (1994)',
 'Transformers: The Movie, The (1986)',
 'Under Si

Vemos que en este caso nos recomienda muchas películas de ese año.

Veamos este otro ejemplo de la película de Pinocho de 1940:

In [27]:
me_gusta = "Pinocchio (1940)"

id_megusta = names_list.index(me_gusta)
corr_recomienda = corrMx[id_megusta]
list(names[(corr_recomienda > .95) & (corr_recomienda < 0.99)])

['Alice in Wonderland (1951)',
 'Beauty and the Beast (1991)',
 'Cinderella (1950)',
 'Mary Poppins (1964)',
 'Old Yeller (1957)',
 'Shaggy Dog, The (1959)',
 'Snow White and the Seven Dwarfs (1937)',
 'Winnie the Pooh and the Blustery Day (1968)']

En este caso las recomendaciones son de diversos años en su mayoría de dibujos animados, aunque no necesariamente, como es el caso de May Popins y  Old Yeller, que son una mezcla de dibujos animados y personas.

### **Fin del ejercicio de la semana 8**